# PART_03. Напишем django app приложение.

### Написание вашего первого приложения Django, часть 3

Этот урок начинается с того места, где закончился урок 2. Мы продолжаем работу над приложением веб-опроса и сосредоточимся на создании публичного интерфейса — «представлений».

`Где получить помощь:`

Если у вас возникли проблемы с изучением этого руководства, перейдите в раздел «Помощь» в разделе «Часто задаваемые вопросы».

# Overview

__Представление (view)__ — это «тип» веб-страницы в вашем приложении Django, который обычно выполняет определенную функцию и имеет определенный шаблон. Например, в приложении блога у вас могут быть следующие представления:

- Домашняя страница блога – отображает несколько последних записей.
- Страница «подробности» записи – страница постоянной ссылки для одной записи.
- Страница архива по годам — отображает все месяцы с записями в данном году.
- Страница архива по месяцам – отображает все дни с записями в данном месяце.
- Страница архива по дням — отображает все записи за данный день.
- Действие комментария – обрабатывает публикацию комментариев к данной записи.

В нашем приложении для опроса у нас будут следующие четыре представления:

1. Страница «указатель» “index” вопросов – отображает несколько последних вопросов.
2. Страница «Подробности» “detail” вопроса — отображает текст вопроса без результатов, но с формой для голосования.
3. Страница «Результаты» “results” вопроса — отображает результаты по конкретному вопросу.
4. Действие голосования – управляет голосованием за определенный вариант в конкретном вопросе.

В Django веб-страницы и другой контент предоставляются посредством представлений. Каждое представление представлено функцией Python (или методом, в случае представлений на основе классов). Django выберет представление, проверив запрошенный URL-адрес (точнее, часть URL-адреса после имени домена).

За время пребывания в сети вы, возможно, встречали таких красавиц, как _`ME2/Sites/dirmod.htm?sid=&type=gen&mod=Core+Pages&gid=A6CD4967199A42D9B65B1B`_. Вам будет приятно узнать, что Django позволяет нам использовать гораздо более элегантные шаблоны URL.

__`Шаблон URL-адреса`__ — это общая форма URL-адреса, например: /newsarchive/<год>/<месяц>/.

Чтобы перейти от URL-адреса к представлению, Django использует так называемые `«URLconfs»`. URLconf сопоставляет шаблоны URL с представлениями.

В этом руководстве представлены базовые инструкции по использованию `URLconfs`, и вы можете обратиться к диспетчеру URL-адресов для получения дополнительной информации.

# Writing more views

Теперь давайте добавим еще несколько просмотров (views) в `mysite/polls/views.py`. Эти взгляды немного отличаются, поскольку они принимают аргумент:

In [ ]:
# mysite/polls/views.py

# ...

def detail(request, question_id):
    return HttpResponse("You're looking at question %s." % question_id)


def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(response % question_id)


def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)

Подключите эти новые представления к модулю `polls.urls`, добавив следующие вызовы path():

In [ ]:
# mysite/polls/urls.py

from django.urls import path
from . import views

urlpatterns = [
    # ex: /polls/
    path("", views.index, name="index"),
    # ex: /polls/5/
    path("<int:question_id>/", views.detail, name="detail"),
    # ex: /polls/5/results/
    path("<int:question_id>/results/", views.results, name="results"),
    # ex: /polls/5/vote/
    path("<int:question_id>/vote/", views.vote, name="vote"),
]

Посмотрите в своем браузере _`«/polls/34/»`_. Он запустит метод Detail() и отобразит любой идентификатор, который вы указали в URL-адресе. Попробуйте также _`«/polls/34/results/»`_ и _`«/polls/34/vote/»`_ — они отобразят результаты-заполнители и страницы голосования.

Когда кто-то запрашивает страницу с вашего веб-сайта, скажем, _`«/polls/34/»`_, Django загрузит модуль Python mysite.urls, поскольку на него указывает параметр `ROOT_URLCONF`. Он находит переменную с именем `urlpatterns` и просматривает шаблоны по порядку. Найдя совпадение в _`polls/`_, он удаляет соответствующий текст ("polls/") и отправляет оставшийся текст - _`"34/"`_ - в URLconf __`polls.urls`__ для дальнейшей обработки. Там он соответствует `<int:question_id>/`, что приводит к вызову представления Detail() следующим образом:
```
detail(request=<HttpRequest object>, question_id=34)
```
Часть `question_id=34` берется из `<int:question_id>`. Использование угловых скобок «захватывает» часть URL-адреса и отправляет ее в качестве аргумента ключевого слова в функцию просмотра. `question_id` определяет имя, которое будет использоваться для идентификации совпадающего шаблона, а часть `int` представляет собой преобразователь, который определяет, какие шаблоны должны соответствовать этой части пути URL. Двоеточие `(:)` разделяет имя конвертера и шаблона.

### Пишите представления, которые действительно что-то делают

Каждое представление отвечает за одно из двух действий: 
1. возврат объекта `HttpResponse`, содержащего содержимое запрошенной страницы, 
2. создание исключения, например `Http404`. Остальное зависит от тебя.

Ваше представление может читать записи из базы данных или нет. Он может использовать систему шаблонов, такую как Django, или стороннюю систему шаблонов Python, или нет. Он может генерировать PDF-файл, выводить XML, создавать ZIP-файл «на лету» и делать все, что вы захотите, используя любые библиотеки Python.

Все, что нужно Django, — это __`HttpResponse`__. Или исключение.

Поскольку это удобно, давайте воспользуемся собственным API базы данных Django, который мы рассмотрели в уроке 2. Вот один из примеров нового представления index(), которое отображает 5 последних вопросов опроса в системе, разделенных запятыми, в зависимости от даты публикации:


In [ ]:
# mysite/polls/views.py

# ...

from django.http import HttpResponse
from .models import Question


def index(request):
    latest_question_list = Question.objects.order_by("-pub_date")[:5]
    output = ", ".join([q.question_text for q in latest_question_list])
    return HttpResponse(output)

# ...

# Leave the rest of the views (detail, results, vote) unchanged

Однако здесь есть проблема: дизайн страницы жестко запрограммирован в представлении. Если вы хотите изменить внешний вид страницы, вам придется отредактировать этот код Python. Итак, давайте воспользуемся системой шаблонов Django, чтобы отделить дизайн от Python, создав шаблон, который может использовать представление.

Сначала создайте каталог под названием `templates` в каталоге `polls`. Django будет искать там шаблоны.

Параметр `TEMPLATES` вашего проекта описывает, как Django будет загружать и отображать шаблоны. Файл настроек по умолчанию настраивает серверную часть `DjangoTemplates`, для параметра `APP_DIRS` которого установлено значение `True`. По соглашению `DjangoTemplates` ищет подкаталог `templates`в каждом из `INSTALLED_APPS.`

В каталоге шаблонов, который вы только что создали, создайте еще один каталог под названием `polls` и внутри него создайте файл `index.html`. Другими словами, ваш шаблон должен находиться по адресу __`mysite/polls/templates/polls/index.html`__. Поскольку загрузчик шаблонов `app_directories` работает, как описано выше, вы можете ссылаться на этот шаблон в Django как `polls/index.html`.

> Пространство имен шаблонов
```
Теперь нам, возможно, удастся поместить наши шаблоны непосредственно в polls/templates (вместо того, чтобы создавать еще один подкаталог опросов), но на самом деле это было бы плохой идеей. Django выберет первый найденный шаблон, имя которого совпадает, и если у вас есть шаблон с таким же именем в другом приложении, Django не сможет различить их. Нам нужно иметь возможность указать Django на правильный путь, и лучший способ обеспечить это — создать для них пространство имен. То есть, поместив эти шаблоны в другой каталог, названный в честь самого приложения.
```
Поместите следующий код в этот шаблон:

In [ ]:
# mysite/polls/templates/polls/index.html

# ...

{% if latest_question_list %}
    <ul>
    {% for question in latest_question_list %}
        <li><a href="/polls/{{ question.id }}/">{{ question.question_text }}</a></li>
    {% endfor %}
    </ul>
{% else %}
    <p>No polls are available.</p>
{% endif %}

# ...

> Примечание
```
Чтобы сделать руководство короче, во всех примерах шаблонов используется неполный HTML. В своих проектах вам следует использовать полные HTML-документы.
```
Теперь давайте обновим наше индексное представление в `polls/views.py`, чтобы использовать шаблон:

In [ ]:
# mysite/polls/views.py

# ...

from django.http import HttpResponse
from django.template import loader

from .models import Question


def index(request):
    latest_question_list = Question.objects.order_by("-pub_date")[:5]
    template = loader.get_template("polls/index.html")
    context = {
        "latest_question_list": latest_question_list,
    }
    return HttpResponse(template.render(context, request))

# ...

Этот код загружает шаблон `polls/index.html` (mysite/polls/templates/polls/index.html) и передает ему контекст. Контекст — это словарь, сопоставляющий имена переменных шаблона с объектами Python.

Загрузите страницу, указав в браузере `/polls/`, и вы увидите маркированный список, содержащий вопрос «Что случилось» из урока 2. Ссылка ведет на страницу с подробными сведениями о вопросе.

### A shortcut: render()

Это очень распространенная идиома: загрузить шаблон, заполнить контекст и вернуть объект `HttpResponse` с результатом визуализированного шаблона. Джанго предоставляет ярлык. Вот полное переписанное представление `index()`:


In [ ]:
# mysite/polls/views.py

# ...

from django.shortcuts import render

from .models import Question


def index(request):
    latest_question_list = Question.objects.order_by("-pub_date")[:5]
    context = {"latest_question_list": latest_question_list}
    return render(request, "polls/index.html", context)

# ...

`Обратите внимание:` как только мы сделали это во всех этих представлениях, нам больше не нужно импортировать `loader` и `HttpResponse` (вы захотите сохранить HttpResponse, если у вас все еще есть методы-заглушки для подробностей, результатов и голосования).

Функция `render()` принимает объект запроса в качестве первого аргумента, имя шаблона в качестве второго аргумента и словарь в качестве необязательного третьего аргумента. Он возвращает объект HttpResponse данного шаблона, отображаемый с заданным контекстом.

### Вызов ошибки 404

Теперь давайте займемся представлением подробностей вопроса — страницей, на которой отображается текст вопроса для данного опроса. Вот views:

In [ ]:
# mysite/polls/views.py

from django.http import Http404
from django.shortcuts import render

from .models import Question


# ...
def detail(request, question_id):
    try:
        question = Question.objects.get(pk=question_id)
    except Question.DoesNotExist:
        raise Http404("Question does not exist")
    return render(request, "polls/detail.html", {"question": question})

`Новая концепция:` представление вызывает исключение `Http404`, если вопрос с запрошенным идентификатором не существует.

Мы обсудим, что вы могли бы поместить в этот шаблон `polls/detail.html` чуть позже, но если вы хотите быстро заставить приведенный выше пример работать, файл, содержащий только: __`polls/templates/polls/detail.html`__

```
{{ question }}
```

поможет вам начать сейчас.

### A shortcut: get_object_or_404()

Очень распространена идиома: использовать `get()` и вызвать `Http404`, если объект не существует. Джанго предоставляет ярлык. Вот переписанное представление Detail():

In [ ]:
# mysite/polls/views.py

from django.shortcuts import get_object_or_404, render

from .models import Question


# ...
def detail(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    return render(request, "polls/detail.html", {"question": question})

Функция `get_object_or_404()` принимает модель Django в качестве первого аргумента и произвольное количество аргументов с ключевым словом, которые передаются в функцию `get()` менеджера модели. Он вызывает `Http404`, если объект не существует.

> Философия
```
Почему мы используем вспомогательную функцию get_object_or_404() вместо автоматического перехвата исключений ObjectDoesNotExist на более высоком уровне или использования API модели для вызова Http404 вместо ObjectDoesNotExist?

Потому что это соединило бы уровень модели со слоем представления. Одна из главных целей дизайна Django — поддерживать слабую связанность. Некоторая контролируемая связь представлена в модуле django.shortcuts.
```
Существует также функция `get_list_or_404()`, которая работает так же, как `get_object_or_404()`, за исключением использования `filter()` вместо `get()`. Он вызывает `Http404`, если список пуст.

### Используйте систему шаблонов (Use the template system)

Вернёмся к представлению `Detail()` нашего приложения для опроса. Учитывая вопрос о контекстной переменной, вот как может выглядеть шаблон _`polls/detail.html`_:


In [ ]:
# mysite/polls/templates/polls/detail.html

<h1>{{ question.question_text }}</h1>
<ul>
{% for choice in question.choice_set.all %}
    <li>{{ choice.choice_text }}</li>
{% endfor %}
</ul>

Система шаблонов использует синтаксис точечного поиска для доступа к атрибутам переменных. В примере `{{ question.question_text }}` сначала Django выполняет поиск в словаре по объекту `question`. В противном случае он пытается выполнить поиск атрибутов, что в данном случае работает. Если бы поиск атрибута не удался, он бы попытался выполнить поиск по индексу списка.

Вызов метода происходит в цикле `{% for %}: question.choice_set.all` интерпретируется как код Python `question.choice_set.all()`, который возвращает итерацию объектов `Choice` и подходит для использования в `{% for % }` tag.

Дополнительную информацию о шаблонах см. в руководстве по шаблонам.

### Удаление жестко запрограммированных URL-адресов в шаблонах

Помните, когда мы писали ссылку на вопрос в шаблоне `polls/index.html`, ссылка была частично жестко закодирована следующим образом:

In [ ]:
<li><a href="/polls/{{ question.id }}/">{{ question.question_text }}</a></li>

Проблема с этим жестко запрограммированным, тесно связанным подходом заключается в том, что становится сложно изменять URL-адреса в проектах с большим количеством шаблонов. Однако, поскольку вы определили аргумент name в функциях `path()` в модуле `polls.urls`, вы можете удалить зависимость от конкретных URL-путей, определенных в ваших конфигурациях URL-адресов, с помощью тега шаблона `{% url %}`:

In [ ]:
<li><a href="{% url 'detail' question.id %}">{{ question.question_text }}</a></li>

Это работает путем поиска определения URL-адреса, указанного в модуле `polls.urls`. Вы можете увидеть, где именно указано URL-имя `«detail»` ниже:

In [ ]:
...
# the 'name' value as called by the {% url %} template tag
path("<int:question_id>/", views.detail, name="detail"),
...

Если вы хотите изменить URL-адрес подробного представления опросов на что-то другое, например на `polls/species/12/`, вместо того, чтобы делать это в шаблоне (или шаблонах), вы должны изменить его в `polls/urls.py`:

In [ ]:
# mysite/polls/urls.py

...
# added the word 'specifics'
path("specifics/<int:question_id>/", views.detail, name="detail"),
...

### Имена URL-адресов пространства имен

В учебном проекте есть только одно приложение — __`polls`__. В реальных проектах Django может быть пять, десять, двадцать приложений и более. Как Django различает имена URL-адресов между ними? Например, приложение для опросов имеет подробное представление, как и приложение для блога, работающее в том же проекте. Как сделать так, чтобы Django знал, какое представление приложения создать для URL-адреса при использовании тега шаблона `{% url %}`?

Ответ заключается в добавлении пространств имен в ваш `URLconf`. В файле `polls/urls.py` добавьте `app_name`, чтобы задать пространство имен приложения:

In [ ]:
# mysite/polls/urls.py

from django.urls import path

from . import views

app_name = "polls"
urlpatterns = [
    path("", views.index, name="index"),
    path("<int:question_id>/", views.detail, name="detail"),
    path("<int:question_id>/results/", views.results, name="results"),
    path("<int:question_id>/vote/", views.vote, name="vote"),
]

Теперь измените шаблон `polls/index.html` на:

In [ ]:
# mysite/polls/templates/polls/index.html

<li><a href="{% url 'detail' question.id %}">{{ question.question_text }}</a></li>

чтобы указать на подробное представление пространства имен:
    

In [ ]:
# mysite/polls/templates/polls/index.html

<li><a href="{% url 'polls:detail' question.id %}">{{ question.question_text }}</a></li>

Когда вы освоитесь с написанием представлений, прочитайте часть 4 этого руководства, чтобы изучить основы обработки форм и общих представлений.

# RESUME:

64. `mysite/polls/views.py` -> создаём три представления detail, results, vote.
65. `mysite/polls/urls.py` -> подключаем представления к модулю polls/urls.py
66. `mysite/polls/views.py` -> пишем новое представление def index()
67. `mysite/polls/templates/polls/` -> создаём новые директории для шаблонов /templates/polls/
68. `mysite/polls/templates/polls/index.html` -> создаём хтмл документ для странички index.html
69. `mysite/polls/views.py` -> обновляем индексное представление def index()
70. `mysite/polls/views.py` -> смотрим ещё один вариант функции представления def index()
71. `mysite/polls/views.py` -> def detail работаем с этим представлением, трай эсепт пробуем. 
72. `mysite/polls/templates/polls/detail.html` -> Создаём шаблон хтмл для созданного ранее представления. detail.html
73. `mysite/polls/urls.py` -> меняем адреса